In [81]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_absolute_error

In [82]:
df_original = pd.read_csv('data/train.csv')
test_original = pd.read_csv('data/test.csv')

In [83]:
df_train, df_test = train_test_split(df_original, test_size=0.2, random_state=0)

In [84]:
x_train = df_train.select_dtypes(exclude='object').drop(columns=['SalePrice', 'Id'])
y_train = df_train['SalePrice']

In [85]:
x_test = df_test.select_dtypes(exclude='object').drop(columns=['SalePrice', 'Id'])
y_test = df_test['SalePrice']

In [205]:
XGBRegressor?

In [189]:
xgb_old = XGBRegressor(random_state=0, n_jobs=-1)
xgb_old.fit(x_train, y_train)

y_test_preds = xgb_old.predict(x_test)
mae = mean_absolute_error(y_test, y_test_preds)
print(mae)

17539.71776005993


In [190]:
xgb = XGBRegressor(random_state=0, n_jobs=-1)

In [258]:
def prange(start, p, count):
    result = [start]
    for step in range(2, count+2):
        result.append(result[-1] * p)
    return result

In [273]:
prange(.75, 1.02, 5)

[0.75, 0.765, 0.7803, 0.795906, 0.81182412, 0.8280606024]

In [278]:
%%time

gird_params = {'objective':['reg:linear'],
              'learning_rate': [.06], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4, 5, 6],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.75],
              'n_estimators': [100]
              }

xgb_gird = GridSearchCV(xgb, gird_params, cv=5, n_jobs = -1, verbose=True)

xgb_gird.fit(x_train, y_train)
print(xgb_gird.best_score_)
print(xgb_gird.best_params_)
xgb_tuned = xgb_gird.best_estimator_

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   25.0s
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:   30.6s finished


0.8744358282655443
{'colsample_bytree': 0.7594920000000001, 'learning_rate': 0.06, 'max_depth': 7, 'min_child_weight': 6, 'n_estimators': 100, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}
CPU times: user 1.21 s, sys: 90.5 ms, total: 1.3 s
Wall time: 31 s


In [194]:
# 0.8744358282655443

In [269]:
%%time
scores = cross_val_score(xgb_tuned, x_train, y_train, scoring='neg_mean_absolute_error', cv=5)
print('Mean Absolute Error %2f' %(-1 * scores.mean()))

Mean Absolute Error 16221.301579
CPU times: user 795 ms, sys: 5.16 ms, total: 800 ms
Wall time: 799 ms


In [270]:
# xgb_tuned.fit(x_train, y_train)
y_test_preds = xgb_tuned.predict(x_test)
mae = mean_absolute_error(y_test, y_test_preds)
print(mae)

17631.06534942209


In [ ]:
# 17539.71776005993